In [3]:
import sys 
from    pathlib import Path 
sys.path.append("/Users/leosheng/Desktop/CMU/capstone/finatars/eval_neo4j")

from database_proxy import graph_client, vector_client 



In [12]:
def random_walk(start_node:str) -> set[str]:
    visited = set() # str: node element_id 
    result = set()
    def dfs(cur:str, cur_doc:list[str], cur_hop, target_hop):
        if cur_hop == target_hop:
            result.add(tuple(sorted(set(cur_doc))))
        else: 
            if cur in visited:
                return 
            else:
                visited.add(cur)
            for triplet in graph_client.get_incoming_outgoing(cur):
                _, relation, nei = triplet
                doc = relation.get("document_id")
                if nei.get("id") not in visited:
                    dfs(nei.get("id"), cur_doc + [doc], cur_hop + 1, target_hop)
    for target_hop in [5]:
        dfs(start_node, [], 0, target_hop)
    return result 


In [13]:
def generate_path(start_nodes:list[str]):
    '''
    start_nodes: str, the name of starting nodes: Nvidia, AMD, ...
    returns 
    chunk_id: list[list[str]]: a list of chunk ids for all paths
    chunk_text:list[list[str]]: a list of chunk texts for all path 
    
    '''
    paths = set()
    for start_node in start_nodes:
        paths |= random_walk(start_node)
    chunk_id = []
    chunk_text = []
    chunk_source = []

    for path in paths:
        docs = set()
        source = set() 
        for id in path:
            doc, s = vector_client.get_document_from_document_id([id])[0]
            docs.add(doc)
            source.add(s)
        chunk_id.append(list(path))
        chunk_text.append(list(docs))
        chunk_source.append(list(source))
    return chunk_id, chunk_text, chunk_source

start_nodes = ["AMD", "Advanced Micro Devices", "Nvidia", "NVIDIA", "Microsoft", "TSM", "Facebook", "meta", "Taiwan Semiconductor Manufacturing"]

chunk_id, chunk_text, chunk_source = generate_path(start_nodes)
print(len(chunk_id))

2170


In [14]:
import pandas as pd 
length = len(chunk_id)
df = pd.DataFrame({"chunk_id":chunk_id, "chunk_source":chunk_source, "chunk_text":chunk_text, "question": ["" for _ in range(length)], "answer": ["" for _ in range(length)]})

In [15]:
import numpy as np
random_index = np.random.choice(len(df), size=120, replace=False)
index_list = [random_index[i*20:(i+1)*20] for i in range(6)]
for i, index in enumerate(index_list):
    data = df.iloc[index]
    data.to_csv(f"qa_data_{i}.csv", index=False)

In [20]:
df.to_csv("qa_data.csv", index=False)

In [10]:
df.iloc[index_list[0]]

,chunk_id,chunk_source,chunk_text,question,answer
748,"[8e049cb4-6276-87d8-d106-bf627865d2a2, b212ddf...",[NVDA vs. TSM: Which Chipmaker Stock is Better...,[Our policy is to include interest and penalti...,,
296,"[3e619c5b-8801-886f-1153-21429e404e1b, 4ebba5d...",[NVDA vs. TSM: Which Chipmaker Stock is Better...,[AMD Powers Hitachi Astemo Next-Generation For...,,
755,"[19857181-dd62-9d08-2747-3331540f0b81, 25c2084...",[Customize Your Meta Avatar With New Body Shap...,[Our policy is to include interest and penalti...,,
1491,"[22f41c3d-ab20-b88e-0433-453a5447809b, 83399df...","[AMD_1, AMD_1A, NVDA_7, Testing Meta Verified ...",[ITEM 1A. RISK FACTORS The risks and uncertain...,,
222,"[0f064687-3f51-7c2c-9ad1-d77b09f66b36, 22f41c3...","[AMD_1, Reasons for the Outperformance of NVID...",[Our policy is to include interest and penalti...,,
1026,"[2113a3c7-6535-c732-d98c-787721cb9f55, 3d9f568...",[NVDA vs. TSM: Which Chipmaker Stock is Better...,[Mark Zuckerberg has spent the last nine month...,,
773,"[7977861a-6481-9ffd-9f83-c5ca05060390, b6b2dfd...",[NVDA vs. TSM: Which Chipmaker Stock is Better...,[NVIDIA Corporation NVDA formally launched a C...,,
110,"[0cdae763-eb8d-2dab-a69c-dc97dc7dc64d, 3e619c5...",[NVDA vs. TSM: Which Chipmaker Stock is Better...,"[Taiwan Semiconductor Manufacturing (TSM), bet...",,
1938,"[00635e78-effa-d59f-fb7a-69457226e94d, 1985718...",[NVDA vs. AMD: Which Chip Stock is the Better ...,[Our policy is to include interest and penalti...,,
1474,"[3e619c5b-8801-886f-1153-21429e404e1b, b78da97...","[AMD_1, AMD_1A, NVDA vs. TSM: Which Chipmaker ...",[Our main area of focus is on delivering the n...,,


In [16]:
from qdrant_client import QdrantClient
client = QdrantClient(vector_client.url, port=vector_client.port)

In [18]:

total = client.search("Documents", query_vector=[0] * 1536, with_payload=True, with_vectors=True, limit=3000)

In [31]:
from util import run_query
system_prompt = """\
Generate a detailed summary for the giving article. Include details like the name of company or product or people. \
I will give you $100 dollars for a good summary!
"""
for i, point in enumerate(total):
    print(i)
    document_id = point.id 
    collection_name = "Documents"
    embedding = point.vector
    payload = point.payload
    document = payload["document"]
    if "company" in payload:
        document = f"This article talks about {payload['company']}: " + document
    summary = run_query(document, system_prompt, "gpt-3.5-turbo-0125")
    payload["summary"] = summary 
    vector_client.insert_payload(collection_name, document_id, payload, embedding)

0
The article discusses the challenges faced by NVDA, a technology company, in predicting demand for its products due to various factors such as the emergence of generative AI models, the impact of cryptocurrency mining on GPU demand, and geopolitical tensions affecting the supply chain. NVDA has introduced new CPU products but faces uncertainty in estimating demand, which could lead to revenue volatility. The company relies on third-party suppliers and developers for its products and services, which poses risks to meeting customer demand and supply chain scalability. Issues such as defects in products, security vulnerabilities, and disruptions in manufacturing processes could impact NVDA's reputation and market share. Overall, NVDA faces uncertainties in forecasting demand and managing its supply chain, which could affect its business operations, revenue, and financial results.
1
The article discusses the implementation of end-to-end encryption across messaging services on Facebook an